In [ ]:
from utils.es_client import get_elasticsearch_client
from utils.list_of_mentions import get_mentions
import pandas as pd
import uuid, numpy as np
from elasticsearch import Elasticsearch
from utils.influence_score import get_influence_score
from dotenv import load_dotenv
from utils.redis_client import redis_client

# Load environment variables
load_dotenv()

# Buat koneksi ke Elasticsearch
es = get_elasticsearch_client()

def create_link_user(df):
    if df['channel'] == 'twitter':
        return f"""https://x.com/{df['username'].strip('@ ')}"""
    if df['channel'] == 'instagram':
        return f"""https://www.instagram.com/{df['username'].strip('@ ')}"""
    if df['channel'] == 'tiktok':
        return f"""https://www.tiktok.com/@{df['username'].strip('@ ')}"""
    if df['channel']=='linkedin':
        return f"""https://www.linkedin.com/in/{df['username'].strip('@ ')}"""
    if df['channel']=='reddit':
        return f"""https://www.reddit.com/{df['username'].strip('@ ')}"""
    
    
    return df['username']
    
def add_negative_driver_flag(df):

    # Ensure all sentiment columns exist
    for sentiment in ['positive', 'negative', 'neutral']:
        col_name = f'sentiment_{sentiment}'
        if col_name not in df.columns:
            df[col_name] = 0
    
    # Create a new column that checks if negative sentiment is the highest
    df['is_negative_driver'] = False
    
    # Compare sentiment counts and set flag if negative is highest
    condition = ((df['sentiment_negative'] > df['sentiment_positive']) & 
                 (df['sentiment_negative'] > df['sentiment_neutral']))
    
    df['is_negative_driver'] = condition
    
    return df

def create_uuid(keyword):
    # Gunakan namespace standar (ada juga untuk URL, DNS, dll)
    namespace = uuid.NAMESPACE_DNS

    return uuid.uuid5(namespace, keyword)

def rule_base_user_category(string, category):
    if pd.isna(string):
        return ''
    
    if 'news' in string.lower():
        return 'News Account'

    return category
    
def search_kol(   owner_id = None,
    project_name = None,
    es_host=None,
    es_username=None,
    es_password=None,
    use_ssl=False,
    verify_certs=False,
    ca_certs=None,
    keywords=None,
    search_keyword=None,
    search_exact_phrases=False,
    case_sensitive=False,
    sentiment=None,
    start_date=None,
    end_date=None,
    date_filter="last 30 days",
    custom_start_date=None,
    custom_end_date=None,
    channels=None,
    importance="all mentions",
    influence_score_min=None,
    influence_score_max=None,
    region=None,
    language=None,
    domain=None):

    # Generate cache key based on all parameters
    cache_key = redis_client.generate_cache_key(
        "kol_overview",
        owner_id=owner_id,
        project_name=project_name,
        keywords=keywords,
        search_keyword=search_keyword,
        search_exact_phrases=search_exact_phrases,
        case_sensitive=case_sensitive,
        start_date=start_date,
        sentiment=sentiment,
        end_date=end_date,
        date_filter=date_filter,
        custom_start_date=custom_start_date,
        custom_end_date=custom_end_date,
        channels=channels,
        importance=importance,
        influence_score_min=influence_score_min,
        influence_score_max=influence_score_max,
        region=region,
        language=language,
        domain=domain
    )

    # Try to get from cache first
    cached_result = redis_client.get(cache_key)
    if cached_result is not None:
        print('Returning cached result')
        return cached_result


    print('get all data mentions')

    if not sentiment:
        sentiment = ['positive','negative','neutral']

    result = get_mentions(
            source= ["issue","user_connections","user_followers","user_influence_score",
                     'user_image_url',"engagement_rate","subscriber",
                 "influence_score","reach_score", "viral_score",
                 "sentiment", "link_post","user_category","username",'channel',
                "votes","likes",'comments','shares','retweets','reports','replies',
                 'views','favorites',"post_created_at"],
            page_size=10000,
            es_host=es_host,    
            es_username=es_username,
            es_password=es_password,
            use_ssl=use_ssl,
            verify_certs=verify_certs,
            ca_certs=ca_certs,
            keywords=keywords,
            search_keyword=search_keyword,
            search_exact_phrases=search_exact_phrases,
            case_sensitive=case_sensitive,
            sentiment=sentiment,
            start_date=start_date,
            end_date=end_date,
            date_filter=date_filter,
            custom_start_date=custom_start_date,
            custom_end_date=custom_end_date,
            channels=channels,
            importance=importance,
            influence_score_min=influence_score_min,
            influence_score_max=influence_score_max,
            region=region,
            language=language,
            domain=domain,
            sort_type = 'popular'
        )

    if not result['data']:
        return []
    else:
        kol = pd.DataFrame(result['data'])

        #-----------------------
        if 'user_category' not in kol:
            kol['user_category']=''

        kol['user_category'] = kol.apply(lambda s: 'News Account' if s['channel'] == 'news' else rule_base_user_category(s['username'], s['user_category']), axis=1)

        kol['user_influence_score'] = kol.apply(lambda s: get_influence_score(s), axis=1)

        #-----------------------            
        for c in set(['user_connections','user_followers','subscriber']) - set(kol.columns):
            kol[c] = 0                   


        kol[['user_followers',"subscriber"]] = kol[['user_followers',"subscriber"]].fillna(0)
        kol['user_category'] = kol['user_category'].fillna('')

        kol['link_user'] = kol.apply(lambda s: create_link_user(s), axis=1)        

        for i in set(['user_connections','user_followers','subscriber']) - set(kol):
            kol[i] = 0

        kol['user_followers'] = kol['user_connections'] + kol['user_followers'] + kol['subscriber']

        # Your groupby with sentiment pivot
        agg_kol = kol.groupby(['link_user']).agg({
            'link_post': 'size',
            'viral_score': 'sum',
            'reach_score': 'sum',
            'channel': 'max',
            'username': 'max',
            'user_image_url':'max',
            'user_followers':'max',
            "engagement_rate":'sum',
            'issue': lambda s: list(set(s)),
            'user_category': 'max',
            'user_influence_score': 'mean'
        })

        # Get sentiment counts per link_user using crosstab
        sentiment_counts = pd.crosstab(kol['link_user'], kol['sentiment'])

        # Rename columns to add 'sentiment_' prefix
        sentiment_counts = sentiment_counts.add_prefix('sentiment_')

        # Join the sentiment counts with the main result
        final_kol = agg_kol.join(sentiment_counts)

        # If any sentiment category is missing, add it with zeros
        for sentiment in ['positive', 'negative', 'neutral']:
            col_name = f'sentiment_{sentiment}'
            if col_name not in final_kol.columns:
                final_kol[col_name] = 0        


        # Apply the function to final_kol
        final_kol = add_negative_driver_flag(final_kol).reset_index()

        list_issue = [j for i in final_kol['issue'] for j in i]

        print('check topic map')
        #check issue mana yang sudah termap dan mana yg belum
        query_body = {
            "_source":["unified_issue","list_issue"],
            "query":{

                "bool": {

                    "must":[

                        {        "match": {"project_name.keyword":project_name}   },
                        {        "terms": {"list_issue.keyword":list_issue}  },


                    ]

                }
            }
        }




        response = es.search(
            index="topic_cluster",
            body=query_body,
            size = 10000
        )
        top_map = [i['_source'] for i in response['hits']['hits']]
        dict_issue = {}
        if top_map:
            print('get map')
            df_map = pd.DataFrame(top_map)
            df_map = df_map.explode('list_issue').drop_duplicates('list_issue')
            
            for _,i in df_map.iterrows():
                dict_issue.update({i['list_issue']:i['unified_issue']})

        final_kol['unified_issue'] = final_kol['issue'].transform(lambda s: list(set([dict_issue.get(i,i) for i in s]))[:5])
        final_kol['user_category'] = final_kol.apply(lambda s: 'News Account' if s['channel']=='news' else s['user_category'], axis=1)
        final_kol.drop('issue', axis=1, inplace=True)
        final_kol['most_viral'] = final_kol['user_influence_score']


        final_kol["share_of_voice"] = (final_kol["link_post"]/final_kol["link_post"].sum())*100
        
        most_negative_kol = final_kol.sort_values(['is_negative_driver','sentiment_negative','most_viral'], ascending = False)[:100]
        most_viral_kol = final_kol.sort_values('most_viral', ascending = False)[:100]
        
        final_kol = pd.concat([most_negative_kol,most_viral_kol]).drop_duplicates('link_user')


        result = final_kol.sort_values(['is_negative_driver','sentiment_negative','most_viral'], ascending = False)[:150].to_dict(orient = 'records')
        
        # Cache the results for 10 minutes
        redis_client.set_with_ttl(cache_key, result, ttl_seconds=600)
        return result


In [ ]:
    {
        "link_user": "poskota.co.id",
        "link_post": 328,
        "viral_score": 25.95794271789944,
        "reach_score": 295.8,
        "channel": "news",
        "username": "poskota.co.id",
        "user_image_url": "https://logo.clearbit.com/poskota.co.id",
        "user_followers": 0.0,
        "engagement_rate": 35957.92366030789,
        "user_category": "News Account",
        "user_influence_score": 6.0,
        "sentiment_negative": 127,
        "sentiment_neutral": 118,
        "sentiment_positive": 83,
        "is_negative_driver": true,
        "unified_issue": [
            "5 legal online loans registered in OJK",
            "Bank Indonesia maintains benchmark interest rate",
            "5 effective ways to overcome stress",
            "Financial Literacy and Consumer Awareness",
            "How to transfer Gopay balance to bank"
        ],
        "most_viral": 6.0,
        "share_of_voice": 3.2800000000000002
    },

In [1]:
username,
link_post -> count
viral_score -> sum
reach_score -> sum
channel -> 1 unique
user_image_url -> 1 unique
user_followers -> max
engagement_rate -> sum
user_category -> 1 unique
user_influence_score -> sum
sentiment -> pecah jadi 3 yaitu sentiment_negative, sentiment_positive, sentiment_neutral
issue -> list unique issue

SyntaxError: invalid syntax (1843880290.py, line 2)

In [2]:
from utils.es_client import get_elasticsearch_client
from utils.es_query_builder import build_elasticsearch_query, get_indices_from_channels, get_date_range
from utils.script_score import script_score
import pandas as pd
import uuid, numpy as np
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
from utils.redis_client import redis_client

# Load environment variables
load_dotenv()

# Buat koneksi ke Elasticsearch
es = get_elasticsearch_client()

def create_link_user(df):
    if df['channel'] == 'twitter':
        return f"""https://x.com/{df['username'].strip('@ ')}"""
    if df['channel'] == 'instagram':
        return f"""https://www.instagram.com/{df['username'].strip('@ ')}"""
    if df['channel'] == 'tiktok':
        return f"""https://www.tiktok.com/@{df['username'].strip('@ ')}"""
    if df['channel']=='linkedin':
        return f"""https://www.linkedin.com/in/{df['username'].strip('@ ')}"""
    if df['channel']=='reddit':
        return f"""https://www.reddit.com/{df['username'].strip('@ ')}"""
    
    return df['username']
    
def add_negative_driver_flag(df):
    # Ensure all sentiment columns exist
    for sentiment in ['positive', 'negative', 'neutral']:
        col_name = f'sentiment_{sentiment}'
        if col_name not in df.columns:
            df[col_name] = 0
    
    # Create a new column that checks if negative sentiment is the highest
    df['is_negative_driver'] = False
    
    # Compare sentiment counts and set flag if negative is highest
    condition = ((df['sentiment_negative'] > df['sentiment_positive']) & 
                 (df['sentiment_negative'] > df['sentiment_neutral']))
    
    df['is_negative_driver'] = condition
    
    return df

def create_uuid(keyword):
    # Gunakan namespace standar (ada juga untuk URL, DNS, dll)
    namespace = uuid.NAMESPACE_DNS
    return uuid.uuid5(namespace, keyword)

def rule_base_user_category(string, category):
    if pd.isna(string):
        return ''
    
    if 'news' in string.lower():
        return 'News Account'

    return category
    
def search_kol(
    owner_id = None,
    project_name = None,
    es_host=None,
    es_username=None,
    es_password=None,
    use_ssl=False,
    verify_certs=False,
    ca_certs=None,
    keywords=None,
    search_keyword=None,
    search_exact_phrases=False,
    case_sensitive=False,
    sentiment=None,
    start_date=None,
    end_date=None,
    date_filter="last 30 days",
    custom_start_date=None,
    custom_end_date=None,
    channels=None,
    importance="all mentions",
    influence_score_min=None,
    influence_score_max=None,
    region=None,
    language=None,
    domain=None):

    # Generate cache key based on all parameters
    cache_key = redis_client.generate_cache_key(
        "kol_overview",
        owner_id=owner_id,
        project_name=project_name,
        keywords=keywords,
        search_keyword=search_keyword,
        search_exact_phrases=search_exact_phrases,
        case_sensitive=case_sensitive,
        start_date=start_date,
        sentiment=sentiment,
        end_date=end_date,
        date_filter=date_filter,
        custom_start_date=custom_start_date,
        custom_end_date=custom_end_date,
        channels=channels,
        importance=importance,
        influence_score_min=influence_score_min,
        influence_score_max=influence_score_max,
        region=region,
        language=language,
        domain=domain
    )

    # Try to get from cache first
    cached_result = redis_client.get(cache_key)
    if cached_result is not None:
        print('Returning cached result')
        return cached_result

    print('get all data mentions using aggregation')

    if not sentiment:
        sentiment = ['positive','negative','neutral']

    # Buat koneksi Elasticsearch
    es_conn = get_elasticsearch_client(
        es_host=es_host,
        es_username=es_username,
        es_password=es_password,
        use_ssl=use_ssl,
        verify_certs=verify_certs,
        ca_certs=ca_certs
    )
    
    if not es_conn:
        return []
    
    # Dapatkan indeks dari channel
    indices = get_indices_from_channels(channels)
    
    if not indices:
        print("Error: Tidak ada indeks yang valid")
        return []
    
    # Dapatkan rentang tanggal jika tidak disediakan
    if not start_date or not end_date:
        start_date, end_date = get_date_range(
            date_filter=date_filter,
            custom_start_date=custom_start_date,
            custom_end_date=custom_end_date
        )

    # Build base query menggunakan es_query_builder
    base_query = build_elasticsearch_query(
        keywords=keywords,
        search_keyword=search_keyword,
        search_exact_phrases=search_exact_phrases,
        case_sensitive=case_sensitive,
        sentiment=sentiment,
        start_date=start_date,
        end_date=end_date,
        importance=importance,
        influence_score_min=influence_score_min,
        influence_score_max=influence_score_max,
        region=region,
        language=language,
        domain=domain,
        size=0  # Untuk aggregation saja
    )

    # Tambahkan aggregation untuk KOL analysis
    aggregation_query = {
        "by_username_channel": {
            "terms": {
                "script": {
                    "source": """
                        String username = doc.containsKey('username') && !doc['username'].empty ? doc['username'].value : 'unknown';
                        String channel = doc.containsKey('channel') && !doc['channel'].empty ? doc['channel'].value : 'unknown';
                        return username + '|' + channel;
                    """
                },
                "size": 1000,  # Increase size to get more KOLs
                "order": [
                    {
                        "user_influence_score_avg": "desc"
                    },
                    {
                        "followers_count": "desc"
                    },
                    {
                        "total_posts": "desc"
                    }
                ]
            },
            "aggs": {
                "username": {
                    "terms": {
                        "field": "username",
                        "size": 1
                    }
                },
                "channel": {
                    "terms": {
                        "field": "channel",
                        "size": 1
                    }
                },
                "followers_count": {
                    "max": {
                        "script": {
                            "source": """
                                // Logic untuk followers dengan fallback - menggunakan max untuk mendapatkan nilai tertinggi per user
                                if (doc.containsKey('user_followers') && !doc['user_followers'].empty) {
                                    return doc['user_followers'].value;
                                } else if (doc.containsKey('user_connections') && !doc['user_connections'].empty) {
                                    return doc['user_connections'].value;
                                } else if (doc.containsKey('subscriber') && !doc['subscriber'].empty) {
                                    return doc['subscriber'].value;
                                } else {
                                    return 0;
                                }
                            """
                        }
                    }
                },
                "total_posts": {
                    "value_count": {
                        "field": "username"
                    }
                },
                "viral_score_sum": {
                    "sum": {
                        "field": "viral_score"
                    }
                },
                "reach_score_sum": {
                    "sum": {
                        "field": "reach_score"
                    }
                },
                "unique_user_image_url": {
                    "terms": {
                        "field": "user_image_url",
                        "size": 1
                    }
                },
                "engagement_rate_sum": {
                    "sum": {
                        "field": "engagement_rate"
                    }
                },
                "unique_user_category": {
                    "terms": {
                        "field": "user_category.keyword",
                        "size": 1
                    }
                },
                "user_influence_score_avg": {
                    "avg": {
                        "script": script_score
                    }
                },
                "sentiment_positive": {
                    "filter": {
                        "term": {
                            "sentiment": "positive"
                        }
                    }
                },
                "sentiment_negative": {
                    "filter": {
                        "term": {
                            "sentiment": "negative"
                        }
                    }
                },
                "sentiment_neutral": {
                    "filter": {
                        "term": {
                            "sentiment": "neutral"
                        }
                    }
                },
                "unique_issues": {
                    "terms": {
                        "field": "issue.keyword",
                        "size": 10
                    }
                }
            }
        }
    }

    # Tambahkan aggregation ke base query
    base_query["aggs"] = aggregation_query

    try:
        import json
        print(json.dumps(base_query, indent=2))
        
        # Execute aggregation query
        response = es_conn.search(
            index=",".join(indices),
            body=base_query
        )
        
        # Process aggregation results
        buckets = response["aggregations"]["by_username_channel"]["buckets"]
        
        if not buckets:
            return []
        
        # Convert aggregation results to DataFrame format
        kol_data = []
        for bucket in buckets:
            username_channel = bucket["key"]
            username = bucket["username"]["buckets"][0]["key"] if bucket["username"]["buckets"] else "unknown"
            channel = bucket["channel"]["buckets"][0]["key"] if bucket["channel"]["buckets"] else "unknown"
            
            # Extract issues list
            issues_list = [issue_bucket["key"] for issue_bucket in bucket["unique_issues"]["buckets"]]
            
            # Get user_category
            user_category = ""
            if bucket["unique_user_category"]["buckets"]:
                user_category = bucket["unique_user_category"]["buckets"][0]["key"]
            
            # Get user_image_url
            user_image_url = ""
            if bucket["unique_user_image_url"]["buckets"]:
                user_image_url = bucket["unique_user_image_url"]["buckets"][0]["key"]
            
            kol_record = {
                'username': username,
                'channel': channel,
                'link_post': bucket["total_posts"]["value"],
                'viral_score': bucket["viral_score_sum"]["value"],
                'reach_score': bucket["reach_score_sum"]["value"],
                'user_image_url': user_image_url,
                'user_followers': bucket["followers_count"]["value"],
                'engagement_rate': bucket["engagement_rate_sum"]["value"],
                'issue': issues_list,
                'user_category': user_category,
                'user_influence_score': bucket["user_influence_score_avg"]["value"],
                'sentiment_positive': bucket["sentiment_positive"]["doc_count"],
                'sentiment_negative': bucket["sentiment_negative"]["doc_count"],
                'sentiment_neutral': bucket["sentiment_neutral"]["doc_count"]
            }
            kol_data.append(kol_record)
        
        # Convert to DataFrame
        final_kol = pd.DataFrame(kol_data)
        
        if final_kol.empty:
            return []

        # Apply business logic transformations
        final_kol['user_category'] = final_kol.apply(lambda s: 'News Account' if s['channel'] == 'news' else rule_base_user_category(s['username'], s['user_category']), axis=1)
        final_kol['link_user'] = final_kol.apply(lambda s: create_link_user(s), axis=1)
        
        # Apply negative driver flag
        final_kol = add_negative_driver_flag(final_kol)
        
        # Get all unique issues for topic mapping
        list_issue = [j for i in final_kol['issue'] for j in i]

        print('check topic map')
        # Check issue mapping (same logic as before)
        query_body = {
            "_source": ["unified_issue", "list_issue"],
            "query": {
                "bool": {
                    "must": [
                        {"match": {"project_name.keyword": project_name}},
                        {"terms": {"list_issue.keyword": list_issue}},
                    ]
                }
            }
        }

        response = es.search(
            index="topic_cluster",
            body=query_body,
            size=10000
        )
        
        top_map = [i['_source'] for i in response['hits']['hits']]
        dict_issue = {}
        if top_map:
            print('get map')
            df_map = pd.DataFrame(top_map)
            df_map = df_map.explode('list_issue').drop_duplicates('list_issue')
            
            for _, i in df_map.iterrows():
                dict_issue.update({i['list_issue']: i['unified_issue']})

        # Apply unified issue mapping
        final_kol['unified_issue'] = final_kol['issue'].transform(lambda s: list(set([dict_issue.get(i, i) for i in s]))[:5])
        final_kol['user_category'] = final_kol.apply(lambda s: 'News Account' if s['channel'] == 'news' else s['user_category'], axis=1)
        final_kol.drop('issue', axis=1, inplace=True)
        final_kol['most_viral'] = final_kol['user_influence_score']

        # Calculate share of voice
        final_kol["share_of_voice"] = (final_kol["link_post"] / final_kol["link_post"].sum()) * 100
        
        # Get top KOLs
        most_negative_kol = final_kol.sort_values(['is_negative_driver', 'sentiment_negative', 'most_viral'], ascending=False)[:100]
        most_viral_kol = final_kol.sort_values('most_viral', ascending=False)[:100]
        
        final_kol = pd.concat([most_negative_kol, most_viral_kol]).drop_duplicates('link_user')

        result = final_kol.sort_values(['is_negative_driver', 'sentiment_negative', 'most_viral'], ascending=False)[:150].to_dict(orient='records')
        
        # Cache the results for 10 minutes
        redis_client.set_with_ttl(cache_key, result, ttl_seconds=600)
        return result
        
    except Exception as e:
        print(f"Error executing aggregation query: {e}")
        return []

------------------connect to redis----------------
Successfully connected to http://34.101.178.71:9200


In [20]:
hasil = search_kol(keywords = ['prabowo','gibran'],
                   project_name='gibran raka',
                   date_filter = 'lat year',
                   owner_id = 5,channels = ['linkedin'])

get all data mentions using aggregation
Successfully connected to http://34.101.178.71:9200
{
  "size": 0,
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "post_created_at": {
              "gte": "2000-01-01",
              "lte": "2025-05-25"
            }
          }
        },
        {
          "bool": {
            "must": [
              {
                "bool": {
                  "should": [
                    {
                      "match": {
                        "post_caption": {
                          "query": "prabowo",
                          "operator": "AND"
                        }
                      }
                    },
                    {
                      "match": {
                        "issue": {
                          "query": "prabowo",
                          "operator": "AND"
                        }
                      }
                    },
                    {
                     

INFO:elastic_transport.transport:POST http://34.101.178.71:9200/linkedin_data/_search [status:200 duration:0.356s]
INFO:elastic_transport.transport:POST http://34.101.178.71:9200/topic_cluster/_search [status:200 duration:0.067s]


check topic map
get map


In [21]:
pd.DataFrame(hasil).sort_values(['share_of_voice'], ascending = False)

,username,channel,link_post,viral_score,reach_score,user_image_url,user_followers,engagement_rate,user_category,user_influence_score,sentiment_positive,sentiment_negative,sentiment_neutral,link_user,is_negative_driver,unified_issue,most_viral,share_of_voice
58,mr-iqbal-masril-djanaik-78992122b,linkedin,11,0.378460,4.054048,https://media.licdn.com/dms/image/v2/D4E03AQFd...,7814.0,626.014488,Human,0.564584,10,0,1,https://www.linkedin.com/in/mr-iqbal-masril-dj...,False,"[Proposal for textile industry consolidation, ...",0.564584,8.730159
0,erudite-risk,linkedin,9,10.071918,0.057000,https://media.licdn.com/dms/image/v2/C4D0BAQHC...,0.0,100.000000,Human,0.193265,3,4,2,https://www.linkedin.com/in/erudite-risk,True,[Flood victim assistance and government respon...,0.193265,7.142857
31,acrosticsasia,linkedin,8,0.740449,0.551429,https://media.licdn.com/dms/image/v2/D560BAQHy...,0.0,225.000000,Human,0.220211,0,1,7,https://www.linkedin.com/in/acrosticsasia,False,"[Challenges facing Indonesia, Danantara's pote...",0.220211,6.349206
66,indonesia-economic-summit-2025,linkedin,3,0.000000,0.000000,https://media.licdn.com/dms/image/v2/D560BAQEd...,0.0,0.000000,Human,0.000000,2,0,1,https://www.linkedin.com/in/indonesia-economic...,False,[Indonesia's efforts to attract private capita...,0.000000,2.380952
1,jimmy-saragi,linkedin,3,0.000000,0.565714,https://media.licdn.com/dms/image/v2/D5603AQFq...,1141.0,216.666664,human,0.622719,0,3,0,https://www.linkedin.com/in/jimmy-saragi,True,"[Inconsistency in Indonesian policy, Inconsist...",0.622719,2.380952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,netmedia101,linkedin,1,0.000000,0.000000,,0.0,0.000000,,0.000000,0,1,0,https://www.linkedin.com/in/netmedia101,True,[Scams using Prabowo deepfakes],0.000000,0.793651
26,michaelwestmedia,linkedin,1,0.000000,0.000000,,0.0,0.000000,,0.000000,0,1,0,https://www.linkedin.com/in/michaelwestmedia,True,[Indonesia's potential one-party state],0.000000,0.793651
25,marketinginteractive,linkedin,1,0.000000,0.000000,,0.0,0.000000,,0.000000,0,1,0,https://www.linkedin.com/in/marketinginteractive,True,[Prabowo's economic policies and protests],0.000000,0.793651
24,kentaro-iwamoto,linkedin,1,0.000000,0.000000,,0.0,0.000000,,0.000000,0,1,0,https://www.linkedin.com/in/kentaro-iwamoto,True,[Investor skepticism towards Danantara],0.000000,0.793651
